# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation. To run this example Julia's `PyCall` package
needs to be able to find an installation of `pymatgen`.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
│   caller = npyinitialize() at numpy.jl:67
└ @ PyCall /home/runner/.julia/packages/PyCall/L0fLP/src/numpy.jl:67


Setup an LDA model and discretize using
a single k-point and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis discretization:
    Ecut                 : 5.0 Ha
    fft_size             : (32, 32, 32), 32768 total points
    kgrid type           : Monkhorst-Pack
    kgrid                : [1, 1, 1]
    num. irred. kpoints  : 1

    Discretized Model(lda_xc_teter93, 3D):
        lattice (in Bohr)    : [0         , 10.2631   , 10.2631   ]
                               [10.2631   , 0         , 10.2631   ]
                               [10.2631   , 10.2631   , 0         ]
        unit cell volume     : 2162.1 Bohr³
    
        atoms                : Si₁₆
        atom potentials      : ElementPsp(Si, psp="hgh/lda/si-q4")
    
        num. electrons       : 64
        spin polarization    : none
        temperature          : 0 Ha
    
        terms                : Kinetic()
                               AtomicLocal()
                               AtomicNonlocal()
                               Ewald()
                               PspCorrection()
                               

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.112619e+02     1.667666e+00
 * time: 0.4822978973388672
     1     1.055433e+01     9.118738e-01
 * time: 1.2437820434570312
     2    -1.181642e+01     1.051866e+00
 * time: 1.3228800296783447
     3    -3.399627e+01     8.508238e-01
 * time: 1.430250883102417
     4    -4.753362e+01     6.288202e-01
 * time: 1.5623500347137451
     5    -5.703422e+01     1.670421e-01
 * time: 1.649662971496582
     6    -5.996041e+01     1.012886e-01
 * time: 1.71331787109375
     7    -6.098776e+01     4.858283e-02
 * time: 1.7769038677215576
     8    -6.135013e+01     5.774975e-02
 * time: 1.8489608764648438
     9    -6.162907e+01     3.269137e-02
 * time: 2.0293569564819336
    10    -6.183087e+01     2.721120e-02
 * time: 2.094251871109009
    11    -6.199550e+01     2.440487e-02
 * time: 2.161911964416504
    12    -6.205845e+01     1.891038e-02
 * time: 2.229367971420288
    13    -6.212710e+01     1.217234e-02
 * time: 2.2940170764923096


In [5]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             25.7671061
    AtomicLocal         -18.8557671
    AtomicNonlocal      14.8522649
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485366 
    Xc                  -19.3336818

    total               -62.261666456753